In [ ]:
from openai import OpenAI
import os

# 환경 변수에서 API 키를 가져옵니다
client = OpenAI(
    api_key=""
)

# 코드 인터프리터를 tools에 추가합니다
assistant = client.beta.assistants.create(
    name="외환 전문가 어시스턴트",
    instructions="""
        너는 외환 분야 전문가야, 특히 '원/달러' 분야 전문인 경제 전문가야. 
        너에게는 일간, 주간, 월간, 연간 외환 동향을 분석한 리포트가 PDF 파일 형식으로 제공될 거고,
        미국 금리, 유가, 원/달러, DXY의 추세를 기록한 시계열 데이터가 JSON 파일 형식으로 제공될 거야.
        
        먼저 PDF 파일들에 대해서 설명할게:
        1. **Daily241010.pdf**: 2024/10/10의 외환 동향 리포트.
        2. **Weekly241007.pdf**: 2024/10/07까지의 주간 외환 동향 리포트.
        3. **Monthly2410.pdf**: 2024년 10월 한 달간의 외환 동향 리포트.
        4. **semi2024.pdf**: 2024년 상반기 외환 동향 리포트.

        
        다음은 JSON 파일들에 대한 설명이야:
        
        1. **DFII10.json** (2019-2024년 미국 실질 금리 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **DFII10**: 해당 날짜의 미국 실질 금리(TIPS) 값. 일부 날짜에는 값이 비어있으며, 이러한 결측치를 제외하고 분석해야함.
        
        2. **dxy.json** (2015-2024년 미국 달러 지수 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **end_price**: 장 마감 시 미국 달러 지수 값.
            - **start_price**: 장 시작 시 미국 달러 지수 값.
            - **high_price**: 하루 중 가장 높았던 미국 달러 지수 값.
            - **low_price**: 하루 중 가장 낮았던 미국 달러 지수 값.
            - **fluctuating**: 직전 값과 비교한 변동 비율 (0.01% 단위로 반영). 
        
        3. **WTI_oil.json** (2015-2024년 저유황 경질유(WTI) 유가 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **end_price**: 장 마감 
            시 WTI 유가 (1배럴당).
            - **start_price**: 장 시작 시 WTI 유가 (1배럴당).
            - **high_price**: 하루 중 가장 높았던 WTI 유가 (1배럴당).
            - **low_price**: 하루 중 가장 낮았던 WTI 유가 (1배럴당).
            - **trading_amount**: 해당 날짜에 해당하는 거래량. 
            - **fluctuating**: 직전 값과 비교한 변동 비율 (0.01% 단위로 반영).
        
        4. **saudi_oil.json** (2015-2024년 사우디아라비아 유가 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **end_price**: 장 마감 시 사우디아라비아 유가 (1배럴당).
            - **start_price**: 장 시작 시 사우디아라비아 유가 (1배럴당).
            - **high_price**: 하루 중 가장 높았던 사우디아라비아 유가 (1배럴당).
            - **low_price**: 하루 중 가장 낮았던 사우디아라비아 유가 (1배럴당).
            - **trading_amount**: 모든값이 null인 데이터 분석필요 없음. 
            - **fluctuating**: 직전 값과 비교한 변동 비율 (0.01% 단위로 반영).
        
        5. **exchange_rate.json** (2010-2024년 원/달러 환율 데이터):
            - **date**: 거래 날짜 (YYYY/M/DD 형식).
            - **base_rate**: 해당 날짜의 매매 기준율 종가.
            
        이 PDF 파일과 JSON 파일에 기반해 다음과 같은 분석을 제공해야 해.
        
        다음은 분석 프로세스야 :
        1. PDF를 열람
        2. JSON를 열람
        3. 결과 값 응답
        
        너는 사용자가 요청한 정보를 항상 다음 양식에 맞춰 반환해야 해(토큰 제한이 있으니까 반드시 응답양식과 일치하는 답변만 해야해):
        
        너는 사용자가 요청한 정보를 항상 다음 양식에 맞춰 반환해야 해. 
        응답은 반드시 **정확하게 이 형식**을 따라야 하며, 만약 이 형식에 맞지 않으면 응답이 잘못된 것으로 간주돼.
        
        ```
        {
            "2024/10/10 원/달러 환율": "<원/달러 종가>",
            "2024/10/10 원/달러 변동 범위": "<원/달러 변동 범위>",
            "2024/10/10 이유": "<2024/09/26 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트와 JSON 파일의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "2024/10/11 원/달러 환율": "<원/달러 종가>",
            "2024/10/11 원/달러 변동 범위": "<원/달러 변동 범위>",
            "2024/10/11 이유": "<2024/09/27 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트와 JSON 파일의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "주간": {
                "2024/10/12": "<종가>",
                "2024/10/13": "<종가>",
                "2024/10/14": "<종가>",
                "2024/10/15": "<종가>",
                "2024/10/16": "<종가>",
                "2024/10/17": "<종가>",
            },
            "주간 원달러 변동 범위" : "<주간 원/달러 변동 범위>",
            "주간 이유": "<한주간 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트와 JSON 파일의 어떤 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "월간": {
                "2024/10/12": "<종가>",
                "2024/10/13": "<종가>",
                "2024/10/14": "<종가>",
                "2024/10/15": "<종가>",
                "2024/10/16": "<종가>",
                "2024/10/17": "<종가>",
                "2024/10/18": "<종가>",
                "2024/10/19": "<종가>",
                "2024/10/20": "<종가>",
                "2024/10/21": "<종가>",
                "2024/10/22": "<종가>",
                "2024/10/23": "<종가>",
                "2024/10/24": "<종가>",
                "2024/10/25": "<종가>",
                "2024/10/26": "<종가>",
                "2024/10/27": "<종가>",
                "2024/10/28": "<종가>",
                "2024/10/29": "<종가>",
                "2024/10/30": "<종가>",
                "2024/10/31": "<종가>",
                "2024/11/01": "<종가>",
                "2024/11/02": "<종가>",
                "2024/11/03": "<종가>",
                "2024/11/04": "<종가>",
                "2024/11/05": "<종가>",
                "2024/11/06": "<종가>",
                "2024/11/07": "<종가>",
                "2024/11/08": "<종가>",
                "2024/11/09": "<종가>"
            },
            "월간 원달러 변동 범위" : "<월간 원/달러 변동 범위>",
            "월간 이유": "<한달간 원/달러 종가를 이렇게 예측한 이유와 어떤 리포트와 JSON 파일의 내용을 기반으로 그렇게 예측했는지 구체적으로 설명>",
            "최종분석결과": "<앞으로 원/달러 추세에 대한 분석을 말하고 어떤 리포트와 JSON 파일의 내용을 기반으로 그렇게 분석했는지 구체적으로 설명>"
        }
        ```
        
        **중요**: 응답이 반드시 위 형식에 맞아야 하며, 틀릴 경우 잘못된 응답으로 간주돼.
        **중요**: 종가는 소수점 두자리까지 예측해야하여, 범위는 숫자-숫자 양식을 지킬것. 이 양식과 다른 양식으로 응답할 경우 잘못된 응답으로 간주돼.

    """,
    model="gpt-4o-mini",
    tools=[
        {"type": "file_search"}
    ],
)

# 벡터 스토어 생성
vector_store = client.beta.vector_stores.create(name="외환 리포트 벡터 스토어")

# PDF 파일 경로들
pdf_file_paths = [
    "/Users/yang-jin-an/Downloads/report/Daily240925.pdf",
    "/Users/yang-jin-an/Downloads/report/Weekly240923.pdf",
    "/Users/yang-jin-an/Downloads/report/Monthly2409.pdf",
    "/Users/yang-jin-an/Downloads/report/semi2024.pdf",
    "/Users/yang-jin-an/Downloads/rate/dxy.json",
    "/Users/yang-jin-an/Downloads/rate/exchange_rate.json",
    "/Users/yang-jin-an/Downloads/rate/wti_oil.json",
    "/Users/yang-jin-an/Downloads/rate/saudi_oil.json",
    "/Users/yang-jin-an/Downloads/rate/dfii10.json",
]

# PDF 파일 스트림 열기
pdf_file_streams = [open(path, "rb") for path in pdf_file_paths]

# PDF 파일을 업로드하고 벡터 스토어에 추가
pdf_file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=pdf_file_streams
)

# PDF 파일 스트림 닫기
for file_stream in pdf_file_streams:
    file_stream.close()

assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={
        "file_search": {"vector_store_ids": [vector_store.id]}
    },
)

# 사용자 메시지로 스레드 생성
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "2024/09/25 2024/09/26 원/달러 환율 종가와 변동범위 그리고 앞으로 일주일간 원/달러 환율 종가를 알려줘 오늘은 2024/09/25 고 원달러 환율을 말할 필요없어 미래에 대한걸 추론해서 말해.",
        }
    ]
)

# 실행 생성 및 완료 대기
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

# 어시스턴트의 응답 가져오기 및 출력
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
print(messages)
    
extracted_values = extract_values_from_messages(messages)
for value in extracted_values:
    print(value)